In [1]:
from general_tools.notebook import gpu_utils
GPU = 2
gpu_utils.setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean
from sklearn.manifold import TSNE

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension, \
                                                 shape_net_core_synth_id_to_category


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions, \
                                           save_pc_prediction_stats, save_stats_of_multi_class_experiments, \
                                           paper_pc_completion_experiment_id_best_epoch
                                                  
from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset, \
                                              embedding_of_entire_dataset_at_tensor

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.data_sets.model_net import pc_loader, classes_to_integers


from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry
from sklearn.svm import LinearSVC

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/orions4-zfs/projects/lins2/Jingwei_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
%reload_ext autoreload

In [5]:
loss = 'chamfer'
# loss = 'emd'
n_pc_samples = 2048
model_net = '40'

In [6]:
experiment_name = '2_three_fc_dec_dropout2048pts_chamfer'
experiment_name = 'four_fc_dec_2048pts_emd'
experiment_name = 'three_fc_dec_longer_2048pts_emd'
experiment_name = '1_three_fc_dec_with_spn_2048pts_chamfer'

print experiment_name

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae/')
train_dir = osp.join(train_dir, experiment_name)

1_three_fc_dec_with_spn_2048pts_chamfer


In [7]:
conf = Conf.load(osp.join(train_dir, 'configuration'))
print conf
conf.n_output = conf.n_input

                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'non_linearity': <function relu at 0x7f8ca565fa28>, 'layer_sizes': [512, 1024, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {}
               experiment_name: 1_three_fc_dec_with_spn_2048pts_chamfer
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     train_dir: /orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/vanilla_ae/1_three_fc_dec_with_spn_20

In [8]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

In [9]:
saved_epochs = read_saved_epochs(conf.train_dir)

In [10]:
#last_epoch = saved_epochs[-1]
ae.restore_model(conf.train_dir, 1000, verbose=True)

Model restored in epoch 1000.


In [11]:
model_net_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Point_Clouds/Model_Net_' + model_net +  '/from_manifold/' 
model_net_dir = osp.join(model_net_dir, str(n_pc_samples))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

In [12]:
pc, model_names, labels = pio.load_crude_point_clouds(train_pc_files, loader=pc_loader, n_threads=10, verbose=True)

train_data = PointCloudDataSet(pc, labels=labels)
pc, model_names, labels = pio.load_crude_point_clouds(test_pc_files, loader=pc_loader, n_threads=10, verbose=True)

test_data = PointCloudDataSet(pc, labels=labels)

9841 pclouds were loaded. They belong in 40 shape-classes.
2467 pclouds were loaded. They belong in 40 shape-classes.


In [13]:
train_feed, train_latent, train_classes = latent_embedding_of_entire_dataset(train_data, ae, conf)
cids = classes_to_integers(int(model_net), train_classes)[1]

In [14]:
print train_latent.shape
print train_classes.shape
print np.unique(train_classes)
train_classes[0]
train_classes
np.histogram(cids, bins=max(cids))

(9841, 1024)
(9841,)
['airplane' 'bathtub' 'bed' 'bench' 'bookshelf' 'bottle' 'bowl' 'car'
 'chair' 'cone' 'cup' 'curtain' 'desk' 'door' 'dresser' 'flower_pot'
 'glass_box' 'guitar' 'keyboard' 'lamp' 'laptop' 'mantel' 'monitor'
 'night_stand' 'person' 'piano' 'plant' 'radio' 'range_hood' 'sink' 'sofa'
 'stairs' 'stool' 'table' 'tent' 'toilet' 'tv_stand' 'vase' 'wardrobe'
 'xbox']


(array([626, 106, 515, 173, 572, 335,  64, 197, 889, 167,  79, 138, 200,
        108, 200, 149, 171, 155, 145, 124, 149, 284, 465, 200,  88, 231,
        239, 104, 115, 128, 680, 124,  90, 392, 163, 344, 267, 475, 190]),
 array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
         22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
         33.,  34.,  35.,  36.,  37.,  38.,  39.]))

In [15]:
""" 
a list of lsvc dicts.
Every lsvc dict is organized as {'C': xxx, 'loss': xxx, 'penalty': xxx, 'dual': xxx, 'max_iter': xxx, 'tol': xxx}    
"""
lsvc_list = []
C = [0.1, 0.5, 1.0, 5.0, 10.0]
Loss = ['squared_hinge', 'hinge']
Penalty = ['l2', 'l1']
Dual = [True, False]
Intercept_scaling = [1, 5]

In [16]:
test_feed, test_latent, test_classes = latent_embedding_of_entire_dataset(test_data, ae, conf)
cids_ = classes_to_integers(int(model_net), test_classes)[1]

In [17]:
# dual is suggested to be False when n_samples>n_features
# dual=False
# loss = 'hinge'
for c in C:
    for loss in Loss:
        for penalty in Penalty:
            for dual in Dual:
                for intercept_scaling in Intercept_scaling:
                    print 'C: %.1f, loss: %s, penalty: %s, dual: %r, intercept scaling: %.1f' % (c, loss, penalty, dual, intercept_scaling)
                    try:
                        curr_lsvc = LinearSVC(C=c, loss=loss, penalty=penalty, dual=dual, intercept_scaling=intercept_scaling)
                        curr_lsvc.fit(train_latent, cids)
                        curr_score = curr_lsvc.score(test_latent, cids_)
                        print 'SCORE: %.4f' % curr_score
                        lsvc_list.append({'lsvc': curr_lsvc, 
                                          'C': c, 
                                          'loss': loss,
                                          'penalty': penalty,
                                          'dual': dual,
                                          'intercept_scaling': intercept_scaling,
                                          'score': curr_score
                                        })
                    except:
                        print 'ERROR: conflict params!'
#lsvc = LinearSVC(C=0.5, penalty='l2', intercept_scaling=1) 
#lsvc.fit(train_latent, cids)

C: 0.1, loss: squared_hinge, penalty: l2, dual: True, intercept scaling: 1.0
SCORE: 0.8245
C: 0.1, loss: squared_hinge, penalty: l2, dual: True, intercept scaling: 5.0
SCORE: 0.8273
C: 0.1, loss: squared_hinge, penalty: l2, dual: False, intercept scaling: 1.0
SCORE: 0.8257
C: 0.1, loss: squared_hinge, penalty: l2, dual: False, intercept scaling: 5.0
SCORE: 0.8277
C: 0.1, loss: squared_hinge, penalty: l1, dual: True, intercept scaling: 1.0
ERROR: conflict params!
C: 0.1, loss: squared_hinge, penalty: l1, dual: True, intercept scaling: 5.0
ERROR: conflict params!
C: 0.1, loss: squared_hinge, penalty: l1, dual: False, intercept scaling: 1.0
SCORE: 0.7669
C: 0.1, loss: squared_hinge, penalty: l1, dual: False, intercept scaling: 5.0
SCORE: 0.7698
C: 0.1, loss: hinge, penalty: l2, dual: True, intercept scaling: 1.0
SCORE: 0.7921
C: 0.1, loss: hinge, penalty: l2, dual: True, intercept scaling: 5.0
SCORE: 0.7953
C: 0.1, loss: hinge, penalty: l2, dual: False, intercept scaling: 1.0
ERROR: confl

In [23]:
curr_lsvc = LinearSVC(C=0.5, loss='squared_hinge', penalty='l2', dual=True, intercept_scaling=3)
curr_lsvc.fit(train_latent, cids)
curr_score = curr_lsvc.score(test_latent, cids_)
print curr_score

0.854073773814


In [18]:
lsvc_list

[{'C': 0.1,
  'dual': True,
  'intercept_scaling': 1,
  'loss': 'squared_hinge',
  'lsvc': LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=1000,
       multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
       verbose=0),
  'penalty': 'l2',
  'score': 0.83826509931090398},
 {'C': 0.1,
  'dual': True,
  'intercept_scaling': 5,
  'loss': 'squared_hinge',
  'lsvc': LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=5, loss='squared_hinge', max_iter=1000,
       multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
       verbose=0),
  'penalty': 'l2',
  'score': 0.83948115119578437},
 {'C': 0.1,
  'dual': False,
  'intercept_scaling': 1,
  'loss': 'squared_hinge',
  'lsvc': LinearSVC(C=0.1, class_weight=None, dual=False, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=1000,
       multi_class='ovr', penalty='l2', rando

In [15]:
## last time without bn
#test_feed, test_latent, test_classes = latent_embedding_of_entire_dataset(test_data, ae, conf)
#cids_ = classes_to_integers(int(model_net), test_classes)[1]
#lsvc.score(test_latent, cids_)

0.83015808674503444

In [17]:
# with bn
test_feed, test_latent, test_classes = latent_embedding_of_entire_dataset(test_data, ae, conf,
                                                                             ae.name+'_1/BatchNormalization_3/batchnorm/add_1:0')
cids_ = classes_to_integers(int(model_net), test_classes)[1]
lsvc.score(test_latent, cids_)

0.82407782732063239

In [15]:
print test_latent.shape

(2467, 512)


In [ ]:
# DO GRID Search of the LinearSVC parameter space: